<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# # Project 3 : Web API and NLP

## Task

**For project 3, the goal is two-fold:

1. Using Pushshift's API, collect posts from any two subreddits.
2. Use NLP to train a classifier on which subreddit a given post came from. This is a binary classification problem.

## Overview

- As part of this project, there are 3 notebooks used.-

1. 01_Data_Gahering - reads the API and runs in a loop with timer to get 100 submissions at a time. We have used gaming section with - boardgames (1400), fallout (1600), rocketleague(1500) and destinythegame (400 rows)
2. 02_NLP - notebook represents EDA and processing for NLP 
3. 03_Classification_Models - Pulls the csv saved to create different classification models.


In [131]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import time
import 

In [ ]:
# call only one url to test
# url='https://api.pushshift.io/reddit/search/submission'

# params = {'subreddit': 'RocketLeague',
#           'size' : 500,
#           'before' : 1616995470 #for df 10
# #           1616995474 - df9
# #           1616996022 - df8
# #           1616996109 - df7
#          } 

In [177]:
#convert the calls into a function so we can loop to get data  
def getredditdata (num, subname, posttime):
    
    df = pd.DataFrame()
    reddit_df = pd.DataFrame()

    for i in range(1, num):
        
        url='https://api.pushshift.io/reddit/search/submission'
        params = {'subreddit': subname,
                  'size' : 100,
                  'before' : posttime
                   }
        push_res = requests.get(url, params)
        push_json = push_res.json()
        time.sleep(5)
        posts = push_json['data']
        df = pd.DataFrame(posts)
        time.sleep(5)
        posttime = df['created_utc'].min()
        dataobj = [reddit_df, df]
        reddit_df = pd.concat(dataobj, axis=0, ignore_index=True)
    
        print(f'Finished pulling {100*i} records from {subname}.')
        
        time.sleep(45)
    
    return reddit_df

In [178]:
#In Reddit - using boardgames to get 1400 records
boardgames_df = getredditdata (15, 'boardgames', int(time.time()))

Finished pulling 100 records from boardgames.
Finished pulling 200 records from boardgames.
Finished pulling 300 records from boardgames.
Finished pulling 400 records from boardgames.
Finished pulling 500 records from boardgames.
Finished pulling 600 records from boardgames.
Finished pulling 700 records from boardgames.
Finished pulling 800 records from boardgames.
Finished pulling 900 records from boardgames.
Finished pulling 1000 records from boardgames.
Finished pulling 1100 records from boardgames.
Finished pulling 1200 records from boardgames.
Finished pulling 1300 records from boardgames.
Finished pulling 1400 records from boardgames.


In [180]:
#In Reddit - using Fallout to get 1600 records
fallout_df = getredditdata (17, 'Fallout', int(time.time()))

Finished pulling 100 records from Fallout.
Finished pulling 200 records from Fallout.
Finished pulling 300 records from Fallout.
Finished pulling 400 records from Fallout.
Finished pulling 500 records from Fallout.
Finished pulling 600 records from Fallout.
Finished pulling 700 records from Fallout.
Finished pulling 800 records from Fallout.
Finished pulling 900 records from Fallout.
Finished pulling 1000 records from Fallout.
Finished pulling 1100 records from Fallout.
Finished pulling 1200 records from Fallout.
Finished pulling 1300 records from Fallout.
Finished pulling 1400 records from Fallout.
Finished pulling 1500 records from Fallout.
Finished pulling 1600 records from Fallout.


In [181]:
#In Reddit - using RocketLeague to get 1500 records
rocketleague_df = getredditdata (16, 'RocketLeague', int(time.time()))

Finished pulling 100 records from RocketLeague.
Finished pulling 200 records from RocketLeague.
Finished pulling 300 records from RocketLeague.
Finished pulling 400 records from RocketLeague.
Finished pulling 500 records from RocketLeague.
Finished pulling 600 records from RocketLeague.
Finished pulling 700 records from RocketLeague.
Finished pulling 800 records from RocketLeague.
Finished pulling 900 records from RocketLeague.
Finished pulling 1000 records from RocketLeague.
Finished pulling 1100 records from RocketLeague.
Finished pulling 1200 records from RocketLeague.
Finished pulling 1300 records from RocketLeague.
Finished pulling 1400 records from RocketLeague.
Finished pulling 1500 records from RocketLeague.


In [182]:
destinygame_df = getredditdata (5, 'DestinyTheGame', int(time.time()))

Finished pulling 100 records from DestinyTheGame.
Finished pulling 200 records from DestinyTheGame.
Finished pulling 300 records from DestinyTheGame.
Finished pulling 400 records from DestinyTheGame.


In [183]:
print(boardgames_df.shape)
print(fallout_df.shape)
print(rocketleague_df.shape)
print(destinygame_df.shape)

(1400, 82)
(1600, 70)
(1500, 81)
(400, 68)


In [184]:
# Getting onlDataFramelftext, title, and subreddit
boardgames_df = boardgames_df[['subreddit', 'selftext', 'title']]
fallout_df = fallout_df[['subreddit', 'selftext', 'title']]
rocketleague_df = rocketleague_df[['subreddit', 'selftext', 'title']]
destinygame_df = destinygame_df[['subreddit', 'selftext', 'title']]

In [192]:
rocketleague_df.isnull().sum()

subreddit    0
selftext     3
title        0
dtype: int64

In [197]:
full_df = pd.DataFrame()
full_df = pd.concat([full_df, boardgames_df])
time.sleep(30)
full_df = pd.concat([full_df, fallout_df])
time.sleep(30)
full_df = pd.concat([full_df, rocketleague_df])
print(f'Merged all the dataframes to get 4500 records.')

In [198]:
full_df.shape

(4500, 3)

In [199]:
full_df.isnull().sum()

subreddit     0
selftext     19
title         0
dtype: int64

In [200]:
full_df.to_csv('./datasets/reddit4500.csv')

In [201]:
full_df1 = pd.concat([full_df, destinygame_df], join="outer")

In [202]:
full_df1.to_csv('./datasets/reddit4900.csv')

In [203]:
full_df1.isnull().sum()

subreddit     0
selftext     20
title         0
dtype: int64